In [74]:
# pip install pybase64
import base64

In [75]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']


"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '../credentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)

In [76]:
# Call the Gmail API
results = service.users().labels().list(userId='me').execute()
labels = results.get('labels', [])

In [89]:
query = "from:(k.hikichi.01.09@gmail.com) subject:(｢キャンペーン商品｣注文の件)"

In [90]:
message_id = service.users().messages().list(userId='me',maxResults=100,q=query).execute()

In [91]:
mail_id = message_id['messages'][0]['id']

In [92]:
message = service.users().messages().get(userId='me', id=mail_id, format='full').execute()

In [93]:
import base64, email
messages = service.users().messages()
msg = message_id['messages'][0]

# (1) rawフォーマットでメールを取得
data = messages.get(userId='me', id=mail_id, format='raw').execute()

In [94]:
raw_data = base64.urlsafe_b64decode(data['raw'])

In [95]:
# (2) Emailを解析する
eml = email.message_from_bytes(raw_data)

In [96]:
body = ""
for part in eml.walk():
    s = part.get_payload(decode=True)
    charset = part.get_content_charset()
    if s:
        body += s.decode(str(charset), errors="replace")
    
    

In [97]:
from bs4 import BeautifulSoup

In [98]:
# (3) 本文を取得
body = ""
for part in eml.walk(): # (4)
    if part.get_content_type() != 'text/plain': # (5)
        continue
    s = part.get_payload(decode=True)
    if isinstance(s, bytes):
        charset = part.get_content_charset() or 'iso-2022-jp' # (6)
        s = s.decode(str(charset), errors="replace")
        body += s            
# (7) 本文を表示
print(body)

テスト株式会社 テスト部
テスト様
お世話になっております。テストです。
ぜひ、下記の通り注文をお願いしたく存じます。
･品名:カタログNo.1234
･数量:100個
･単価:1,800円
･納期:1月12日
万が一納期が難しい場合は、
あらかじめご相談いただきますようお願い申し上げます。



In [99]:
soup = BeautifulSoup(body)

/Users/kokihikichi/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/kokihikichi/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [100]:
soup.text.strip().replace('\n','').replace('\r', '')

'テスト株式会社 テスト部テスト様お世話になっております。テストです。ぜひ、下記の通り注文をお願いしたく存じます。･品名:カタログNo.1234･数量:100個･単価:1,800円･納期:1月12日万が一納期が難しい場合は、あらかじめご相談いただきますようお願い申し上げます。'

In [12]:
with open('./sample_seikyusho.docx', 'rb') as f:
    document = f.read()

In [13]:
document

b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00>\x00\x03\x00\xfe\xff\t\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00k\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00m\x00\x00\x00\x01\x00\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00j\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff

In [19]:
from docx import Document

# 既存ファイルの読み出し
document = Document('koki.docx')

PackageNotFoundError: Package not found at 'koki.docx'

In [18]:
document = docx.Document('./koki.docx')

PackageNotFoundError: Package not found at './koki.docx'